In [1]:
import pandas as pd
import os
import glob
import numpy as np
import strsimpy
print('setup done!')

setup done!


In [2]:
path = os.getcwd()
path = os.path.join(path, 'data/*.csv')
csv_files = glob.glob(path)

In [3]:
df = pd.read_csv(csv_files[0])
for i, file in enumerate(csv_files):
    if i == 0:
        continue
    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp], axis=0)


In [4]:
len(df)

2576

In [4]:
df = df.reset_index(drop=True)

In [5]:
arr = df.values.tolist()

In [6]:
arr[0]

[0,
 'IPHONE 14 PRO MAX',
 'https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang',
 6,
 128,
 31790000,
 '2022-11-27 15:48:10.402643',
 'Yellow',
 1,
 'https://www.xtmobile.vn/vnt_upload/product/10_2022/thumbs/600_iphone14promax_3.jpg']

In [7]:
levenshtein  = strsimpy.NormalizedLevenshtein()

def levenshtein_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 - levenshtein.distance(x,y)

def difference_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 if x==y else 0

In [8]:
def clust_filter(arr):
    flag = [False]*len(arr)
    cluster  = []
    for i in range(0,len(arr)-1):
        if (flag[i]==False):
            list_id = []
            for j in range(i + 1, len(arr)):
                if (flag[j]==False):
                    name = levenshtein_score(arr[i][1],arr[j][1])
                    # cpu = difference_score(arr[i][2],arr[j][2])
                    ram = difference_score(arr[i][2],arr[j][2])
                    rom = difference_score(arr[i][3],arr[j][3])
                    color = difference_score(arr[i][6],arr[j][6])
                    score = 0.55 * name + 0.2 * ram + 0.2 * rom + 0.05 * color
                    if (score > 0.95):
                        flag[j] = True
                        list_id.append(j)
            arr[i].append(list_id)
            cluster.append(arr[i])
    return cluster

In [9]:
cluster = clust_filter(arr)

In [10]:
newDf = pd.DataFrame(cluster, columns=['id', 'name', 'url', 'ram', 'rom', 'price', 'createdDate', 'color', 'status', 'url_img', 'list_id'])

In [11]:
newDf.head()

,id,name,url,ram,rom,price,createdDate,color,status,url_img,list_id
0,0,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chin...,6,128,31790000,2022-11-27 15:48:10.402643,Yellow,1,https://www.xtmobile.vn/vnt_upload/product/10_...,"[1, 2, 3]"
1,4,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-256gb,6,256,36290000,2022-11-27 15:48:10.402643,Yellow,1,https://www.xtmobile.vn/vnt_upload/product/10_...,"[5, 6, 7, 211, 212, 213, 214, 584, 585, 586, 587]"
2,8,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-512gb,6,512,43990000,2022-11-27 15:48:10.402643,Silver,1,https://www.xtmobile.vn/vnt_upload/product/10_...,"[9, 10, 11, 219, 220, 221, 222, 580, 581, 582,..."
3,12,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-1tb,6,1024,45790000,2022-11-27 15:48:10.402643,Black,1,https://www.xtmobile.vn/vnt_upload/product/10_...,"[13, 14, 15, 223, 224, 225, 226, 592, 593, 594..."
4,16,IPHONE 13,https://www.xtmobile.vn/iphone-13-128gb,4,128,18690000,2022-11-27 15:48:10.402643,Black,1,https://www.xtmobile.vn/vnt_upload/product/04_...,"[17, 18, 19, 20, 21]"


In [12]:
newDf.to_json('./result.json')